In [0]:
%tensorflow_version 1.x

import numpy as np
import keras
from keras.datasets import mnist
import sys
from scipy.stats import entropy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, SpatialDropout2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.regularizers import l2
from keras import backend as K

from google.colab import drive
drive.mount("/content/gdrive")

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def apply_dropout_masks(a,b):
    return np.squeeze(np.multiply(a,b))

def multi_mask_predict(layer_fn, multi_mask_input):
    """Applies Keras model layers to multiple arrays of layer inputs for all points across J fixed dropout masks
    #Arguments
        layer_fn: keras backend function applying the mapping corresponding to keras model layers
        multi_mask_input: numpy array of layer inputs for all points across J fixed dropout masks
    #Returns
        predictions from layer_fn applied to multi_mask_input as a numpy array
    """
    layer_output = []
    for mask_num in range(multi_mask_input.shape[0]):
        layer_output.append(layer_fn((multi_mask_input[mask_num], 1)))
    return np.array(layer_output)


def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean and variance of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns
        Mean and variance of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0]))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:] = predictions.flatten()
    prediction = result.mean(axis=0)
    var = result.var(axis=0)
    return [prediction,var]

def run_model (X_train, y_train):
    """Initializes and trains a model from scratch on the given training data, returning test MAE and MSE along with Keras model
    #Arguments
        X_train: training model inputs
        y_train: training model outputs
    #Returns
        Test MAE and MSE, and Keras model
    """
    nb_pool = 3
    dropout_prob = 0.5
    nb_filters = 50
    nb_conv_init = 4 
    nb_conv_agg = 3
    nb_strides_init = 3
    nb_strides_agg = 2
    img_rows = 200
    img_cols = 200
    Weight_Decay = 2.0/len(y_train)

    #Model v2
    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv_init,  strides=nb_strides_agg, data_format="channels_last", input_shape=(img_rows, img_cols,3)))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv_agg, strides=nb_strides_agg, data_format="channels_last"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_last"))

    model.add(Convolution2D(nb_filters*2, nb_conv_agg, strides=1, data_format="channels_last"))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters*2, nb_conv_agg, strides=1, data_format="channels_last"))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool), data_format="channels_last"))
    model.add(Dropout(dropout_prob))

    model.add(Flatten())
    model.add(Dense(150, W_regularizer=l2(Weight_Decay)))
    model.add(Activation('relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(1, W_regularizer=l2(Weight_Decay)))
    model.compile(loss='mean_squared_error', optimizer='adam')

    #Training model
    hist = model.fit(X_train, y_train, batch_size=128, epochs=250, verbose=0)
    #Evaluating model on test data
    f = K.function([model.layers[0].input, K.learning_phase()], 
               [model.layers[-1].output])
    prediction_results = predict_with_uncertainty(f, X_test, n_iter=300)
    predicted_mean = prediction_results[0]
    return [np.mean(np.abs(predicted_mean - y_test)), np.mean(np.square(predicted_mean - y_test)), model]


In [0]:
def fixed_mask_forward_pass(model, forward_pass_input, num_masks, dropout_prob, conv_masks, dense_masks):
      """Makes model predictions with J dropout masks that are fixed across points to enable estimation of Var(Y_{sample})
    Function is specific to the given Keras model.
    #Arguments
        model: keras model
        forward_pass_input: X_{sample}
        num_masks: J, the number of dropout masks being used in estimation of Var(Y_{sample}) and calculation of the EI acquisition function
        dropout_prob: dropout probability 
        conv_masks: first set of dropout masks, applied after second MaxPooling2D layer
        dense_masks: second set of dropout masks, applied after the first Dense layer
    #Returns
        MC dropout predictions across sample points enabling estimation of Var(Y_{sample}), i.e. \hat{Y}_{sample}
    """
    # Functions to retrieve output of intermediate layers
    # Needed for manual implementation of fixed dropout masks 
    # across all data points
    conv = K.function([model.layers[0].input, K.learning_phase()],
                    [model.layers[9].output])

    dense_1 = K.function([model.layers[11].input, K.learning_phase()],
                   [model.layers[13].output])

    dense_2 = K.function([model.layers[15].input, K.learning_phase()],
                   [model.layers[15].output])
    conv_output = np.array(conv((forward_pass_input, 1)))
    dense_1_input = apply_dropout_masks(conv_output, conv_masks)
    dense_1_output = multi_mask_predict(dense_1, dense_1_input)
    dense_2_input = apply_dropout_masks(dense_1_output, dense_masks)
    dense_2_output = np.squeeze(multi_mask_predict(dense_2, dense_2_input))
    return dense_2_output

def ei_acquisition_fn_model_var (univ_covariance, num_pool_samples, num_training_samples, batch_size):
    """Given Var(Y_{sample}), applies batch-mode EI active learning to query points
    #Arguments
        univ_covariance: Var(Y_{sample})
        num_pool_samples: number of pool points in D_{sample}
        num_training_samples: number of training points in D_{sample}
        batch_size: number of queried points per batch
    #Returns
        the indices of queried pool points as they are arranged in univ_covariance
    """
    acq_ind = []
    for acq_num in range(batch_size):
        all_acq_values = np.zeros(num_pool_samples)
        for new_pt_ind in range(num_pool_samples):
            covariance_vector = univ_covariance[num_training_samples+new_pt_ind,:]
            all_acq_values[new_pt_ind] = np.sum(np.square(covariance_vector))/(univ_covariance[num_training_samples+new_pt_ind, num_training_samples+new_pt_ind])
        sorted_top_ind = np.flip(np.argsort(all_acq_values))
        found_new_ind = False
        top_ind_ctr = -1
        while (found_new_ind == False):
            top_ind_ctr += 1
            new_top_ind = sorted_top_ind[top_ind_ctr]
            if new_top_ind not in acq_ind:
                acq_ind.append(new_top_ind)
                found_new_ind = True
        top_cov_vector = np.expand_dims(univ_covariance[num_training_samples+acq_ind[-1], :], axis=1)
        univ_covariance = univ_covariance - np.matmul(top_cov_vector, top_cov_vector.T)/univ_covariance[num_training_samples+acq_ind[-1], num_training_samples+acq_ind[-1]]
    return acq_ind  

def get_acquisition_fn (model, X_train, X_cand, num_masks, tau_inverse, batch_size, dropout_prob):
    """Given sample points, generates J fixed-mask predictions across all sample points, calculates Var(Y_{sample}), and runs batch-mode EI acquisition
    #Arguments
      model: keras model
      X_train: numpy array of sample points that are in the training set
      X_cand: numpy array of sample points that are in the pool
      num_masks: J, the number of dropout masks being used in estimation of Var(Y_{sample}) and calculation of the EI acquisition function
      tau_inverse: inverse model precision hyperparameter
      batch_size: number of queried points per batch
      dropout_prob: dropout probability 
    #Returns
      The indices of pool points queried by batch-mode EI as they are arranged in X_{cand}
    """
    forward_pass_input = np.concatenate((X_train, X_cand))
    forward_pass_output = np.zeros((len(X_train)+len(X_cand), num_masks))
    #Generating J fixed masks to be used in generating \hat{Y}_{sample}
    conv_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 6, 6, 100)), p=[dropout_prob, 1-dropout_prob])
    dense_masks = 1/(1-dropout_prob)*np.random.choice(2, size=((num_masks, 1, 1 , 150)), p=[dropout_prob, 1-dropout_prob])
    #Obtaining J fixed mask predictions on D_{sample} in 2500 point chunks (to avoid exceeding memory limits)
    last_point_ind = 0
    processing_size = 2500
    while last_point_ind < len(X_train) + len(X_cand):
      if last_point_ind+2500<len(X_train) + len(X_cand):
        forward_pass_chunk = forward_pass_input[last_point_ind:last_point_ind+2500]
        forward_pass_output[last_point_ind:last_point_ind+2500] = fixed_mask_forward_pass(model, forward_pass_chunk, num_masks, dropout_prob, conv_masks, dense_masks).T
      else:
        forward_pass_chunk = forward_pass_input[last_point_ind:]
        forward_pass_output[last_point_ind:] = fixed_mask_forward_pass(model, forward_pass_chunk, num_masks, dropout_prob, conv_masks, dense_masks).T
      last_point_ind += 2500
    output_covariance = np.cov(forward_pass_output)
    print('Avg output variance ' + str(np.mean(output_covariance.diagonal())))
    final_output_covariance = output_covariance + (tau_inverse * np.identity(output_covariance.shape[0]))
    return ei_acquisition_fn_model_var(final_output_covariance, X_cand.shape[0], X_train.shape[0], batch_size)


In [0]:
#Active Learning Experiment Parameters/Settings
num_acquisitions = 2000
batch_size = 25
num_candidates=5000
num_masks = 50
mse_dropout_iterations = 300
num_epochs = 250
size_train = 150
num_experiments = 3
dropout_prob = 0.5

#Training batch size (not active learning batch size)
batchsize=128

#Setting tau inverse to 10% of initial average predicted dropout variance on validation set
tau_inverse = 0.1 * 55.477



In [0]:
#Data Loading
data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/UTKFace/"
trind_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/UTKFace/ALScripts_StartTrainSize150/"

X_train_all = np.load(data_path+"X_train.npy")
y_train_all = np.load(data_path+"y_train.npy")
X_test = np.load(data_path+"X_test.npy")
y_test = np.load(data_path+"y_test.npy")


In [0]:
results_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/UTKFace/AL_Results_Start_150/"

#Iterating across active learning experiments, each of which starts with a different initial training set
for e in range(0,1):
  train_data_indices = list(np.load(trind_path+"trainindices"+str(e+1)+".npy"))
  mae_file = "EITauInv"+str(tau_inverse)+"MAEBS"+str(batch_size)+"_Ind"+str(e+1)+".npy"
  mse_file = "EITauInv"+str(tau_inverse)+"MSEBS"+str(batch_size)+"_Ind"+str(e+1)+".npy"
  ind_file = "EITauInv"+str(tau_inverse)+"IndBS"+str(batch_size)+"_Ind"+str(e+1)+".npy"
  #train_data_indices = list(np.load(results_path+ind_file))
  pool_indices = [i for i in range(y_train_all.shape[0]) if i not in train_data_indices]
  #exp_mse = list(np.load(results_path+mse_file))
  #exp_mae = list(np.load(results_path+mae_file))
  exp_mse = []
  exp_mae = []

  #Training initial model and evaluating on test data
  [mae, mse, model] = run_model(X_train_all[train_data_indices,:,:,:], y_train_all[train_data_indices])
  exp_mse.append(mse)
  exp_mae.append(mae)
  num_acquisitions = num_acquisitions - batch_size * (len(exp_mse) - 1)
  print('Initial MSE: ' + str(exp_mse))

  #Acquisition Loop
  for acq in range(num_acquisitions//batch_size):
      #Selecting sample points
      sample_indices = np.random.choice(len(pool_indices)+len(train_data_indices), int(num_candidates*1.2), replace=False)
      pool_sample_ratio = len(sample_indices[sample_indices >= len(train_data_indices)])/num_candidates
      train_sample_size = len(sample_indices[sample_indices < len(train_data_indices)])
      sample_indices_train = (sample_indices[sample_indices < len(train_data_indices)])[0:int(train_sample_size//pool_sample_ratio)]
      sample_indices_pool = (sample_indices[sample_indices >= len(train_data_indices)])[0:num_candidates]
      X_train_fn = X_train_all[train_data_indices][sample_indices_train]
      X_pool_fn = np.concatenate((X_train_all[train_data_indices], X_train_all[pool_indices]))[sample_indices_pool]
      #Running EI Acquisition
      acq_fn_results = get_acquisition_fn(model, X_train_fn, X_pool_fn, num_masks, tau_inverse, batch_size, dropout_prob)
      acq_fn_ind = acq_fn_results
      acq_ind_ind = np.subtract(sample_indices_pool[acq_fn_ind], len(train_data_indices))
      acq_ind = np.array(pool_indices)[acq_ind_ind]
      #Adding queried points to training set
      for data_ind in acq_ind:
        train_data_indices.append(data_ind)
        pool_indices.remove(data_ind)    
      #Retraining model, calculating MSE/MAE, and saving results
      [mae, mse, model] = run_model(X_train_all[train_data_indices,:,:,:], y_train_all[train_data_indices])
      exp_mse.append(mse)
      exp_mae.append(mae)
      print('Len Training Ind: ' + str(len(train_data_indices)) + ', last MSE: ' + str(exp_mse[-1]))
      np.save(results_path+ind_file, np.array(train_data_indices))
      np.save(results_path+mse_file, np.array(exp_mse))
      np.save(results_path+mae_file, np.array(exp_mae))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`



Initial MSE: [428.5975078486125]
Avg output variance 87.65358011705742


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


Len Training Ind: 175, last MSE: 264.7505624458649
Avg output variance 36.488991777764916


...

Len Training Ind: 2100, last MSE: 133.14933026934875
Avg output variance 36.1789867665455


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


Len Training Ind: 2125, last MSE: 128.1708194003817
Avg output variance 37.6174591345793


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


Len Training Ind: 2150, last MSE: 139.61181638373847
